In [1]:
import numpy as np


In [34]:
import requests

url_test = 'https://oda.ft.dk/api/Afstemning?$expand=Sagstrin/Sag&$filter=vedtaget eq true&$inlinecount=allpages'

r = requests.get(url_test)

sager = r.json()

In [96]:
#This function takes dict, extracts the next link, gets the new dict and appends it to the old dict

def get_all_pages(url):

    r = requests.get(url)

    sager = r.json()
    prev = r.json()

    while 'odata.nextLink' in prev:

        r = requests.get(prev['odata.nextLink'])
        sager['value'] += r.json()['value']
        prev = r.json()

    return sager

In [97]:
sager = get_all_pages(url_test)

In [107]:
test = [len(sager['value'][i].keys()) for i in range(len(sager['value']))]